<a href="https://colab.research.google.com/github/pcpiscator/2T2021/blob/main/PlanTrab%20_TCE_PB_Tratado.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install mpl_finance

In [2]:
# bibliotecas para análise de dados
import numpy as np
import pandas as pd
# bibliotecas para gráficos 
import matplotlib.pyplot as plt
import plotly.graph_objs as go
import plotly.offline as py
pd.options.display.max_columns = None
pd.options.display.max_rows = None
pd.options.display.float_format = '{:.2f}'.format
# bibliotecas para visualização
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import matplotlib.dates as mpl_dates
import sklearn
# configuração para ignorar mensagens de advertência (warning). 
import warnings
warnings.filterwarnings('ignore')


plt.style.use('ggplot')

In [3]:
data = pd.read_csv('/content/PlanTrab.csv', sep=';', encoding='latin_1', low_memory=False)

In [ ]:
#Criação de uma base com os dados originais não tratados para eventual necessidade futura
base_original = data

In [4]:
data.columns

Index(['protocolo', 'cd_ugestora', 'valor_licitado', 'ano_homologacao',
       'cnpj_proponente', 'valor_proposta', 'situacao_proposta'],
      dtype='object')

In [7]:
data['protocolo']  = data['protocolo'].str.replace("Doc","")
data['protocolo']  = data['protocolo'].str.replace("Proc","")

In [8]:
data.head(5)

,protocolo,cd_ugestora,valor_licitado,ano_homologacao,cnpj_proponente,valor_proposta,situacao_proposta
0,. 09739/14,201114,14627450,2014,"2,02009E+12",14777359,Perdedora
1,. 09739/14,201114,14627450,2014,"1,20681E+13",14777359,Perdedora
2,. 09739/14,201114,14627450,2014,"1,12684E+13",14627450,Vencedora
3,. 09741/14,201114,2250010,2014,"4,15777E+13",2291900,Perdedora
4,. 09741/14,201114,2250010,2014,"2,02009E+12",2257576,Perdedora


In [ ]:
#retiramos colunas desnecessárias na análise
data2 = data.drop(['nome_municipio','jurisdicionado_id','nome_jurisdicionado','nome_tipo_jurisdicionado','nome_tipo_administracao_jurisdicionado',
                   'nome_esfera_jurisdicionado','objeto_licitacao','valor_estimado_licitacao','data_homologacao_licitacao','situacao_fracassada_licitacao','nome_proponente','nome_estagio_processual_licitacao',
                   'nome_setor_atual_licitacao', 'url'],axis=1)

#renomeamos algumas colunas
data2.rename(columns={'protocolo_licitacao':'protocolo','nome_modalidade_licitacao': 'modalidade',
                      'valor_licitado_licitacao':'valor_licitado','ano_homologacao_licitacao':'ano_homologacao'}, inplace=True)

#eliminamos todas as linhas que tenham valores nulos
data3 = data2.dropna(subset = ['protocolo', 'numero_licitacao', 'modalidade','cd_ugestora','valor_licitado','ano_homologacao', 'cpf_cnpj_proponente', 'valor_proposta', 'situacao_proposta'])

#Eliminamos todas que não forem Convite, Tomada de Preços e Dispensa (8.666)
data3['ano_homologacao'] = data3.ano_homologacao.astype(int)
data3['valor_licitado'] = data3.valor_licitado.astype(int)
data4 =data3.loc[ (data3.modalidade == 'Convite') | (data3.modalidade == 'Tomada de Preços')
| (data3.modalidade == 'Art. 24 - Lei 8.666/93') ]

#Eliminamos todas as licitações até 2018 (inclusive)
data5 =data4.loc[ (data4.ano_homologacao > 2018)]

#Conversão do tipo do atributo "valor_proposta" de str para int
data5['valor_proposta'] = data5['valor_proposta'].astype(str).astype(int)

#Transformo a informação da coluna "situacao_proposta" em valores de 0 e 1
data5['situacao_proposta'] = data5['situacao_proposta'].map({'Vencedora': 1, 'Perdedora': 0})

#Elimina todos os campos da variávem "cpf_cnpj_proponente" com comprimento menor que 12 ou seja, todos os CPF's
data6 = data5[data5['cpf_cnpj_proponente'].str.len() > 11]

#Renomeia a coluna "cpf_cnpj_proponente" para "cnpj_proponente"
data6.rename(columns={'cpf_cnpj_proponente': 'cnpj_proponente'}, inplace=True)

#Conversão do tipo de atributo "cnpj_proponente" de str para int
data6['cnpj_proponente'] = data6['cnpj_proponente'].astype(str).astype(int)

#Cria uma coluna chamada ident_licit, inicialmente vazia e que receberá os valores da coluna protocolo alterados e a coloca na primeira posição
data6["ident_licit"] = ""
data7=data6.reindex(columns=['ident_licit','protocolo', 'numero_licitacao', 'modalidade', 'cd_ugestora','valor_licitado', 'ano_homologacao', 'cnpj_proponente',
       'valor_proposta', 'situacao_proposta'])




In [9]:
#data7.columns

In [ ]:
#Eliminação dos caracteres especiais do atributo "protocolo"
data7['protocolo']  = data7['protocolo'].str.replace(r'\W',"")

#Eliminação das expressões "Doc. " e Prot. " do atributo "protocolo"
data7['protocolo']  = data7['protocolo'].str.replace("Doc","")
data7['protocolo']  = data7['protocolo'].str.replace("Proc","")

#A coluna "ident_licit" recebe os valores da coluna "protocolo" e esta é eliminada juntamente com a coluna "numero_licitacao"
data7['ident_licit']  = data7['protocolo']
data7 = data7.drop(['protocolo','numero_licitacao'], axis = 1)

#Conversão do tipo do atributo "idente_licit" para inteiro
data7['ident_licit'] = data7['ident_licit'].astype(str).astype(int)


In [10]:
data.columns

Index(['protocolo', 'cd_ugestora', 'valor_licitado', 'ano_homologacao',
       'cnpj_proponente', 'valor_proposta', 'situacao_proposta'],
      dtype='object')

In [11]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4008 entries, 0 to 4007
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   protocolo          4008 non-null   object
 1   cd_ugestora        4008 non-null   int64 
 2   valor_licitado     4008 non-null   object
 3   ano_homologacao    4008 non-null   int64 
 4   cnpj_proponente    4008 non-null   object
 5   valor_proposta     4008 non-null   object
 6   situacao_proposta  4008 non-null   object
dtypes: int64(2), object(5)
memory usage: 219.3+ KB


In [13]:
#Conversão do tipo do atributo "valor_proposta" de str para int
data['valor_proposta'] = data['valor_proposta'].astype(str).astype(int)

ValueError: ignored

In [ ]:
#Criação de uma Matriz de Correlação entre os atributos numéricos
correlacao = data7.corr()
correlacao.style.background_gradient(cmap='Blues')


,ident_licit,cd_ugestora,valor_licitado,ano_homologacao,cnpj_proponente,valor_proposta,situacao_proposta
ident_licit,1.000000,-0.069486,-0.266363,0.057333,0.082816,-0.250104,0.011288
cd_ugestora,-0.069486,1.000000,0.195906,-0.023885,-0.047942,0.202046,-0.092268
valor_licitado,-0.266363,0.195906,1.000000,0.021372,-0.137515,0.966956,-0.164775
ano_homologacao,0.057333,-0.023885,0.021372,1.000000,0.100667,0.021895,-0.047264
cnpj_proponente,0.082816,-0.047942,-0.137515,0.100667,1.000000,-0.136666,-0.003675
valor_proposta,-0.250104,0.202046,0.966956,0.021895,-0.136666,1.000000,-0.213280
situacao_proposta,0.011288,-0.092268,-0.164775,-0.047264,-0.003675,-0.213280,1.000000


In [ ]:
#Geração, no GDrive, de um arquivo csv com as alterações efetuadas.
data7.to_csv('/content/drive/MyDrive/00 - Pós Ciência de Dados/99 - TCC/PROJETO 00 - TCC Grupo/01 - Bases Candidatas/02 - TCE PB - Experiência com o Colab/data7.CSV', sep=',', encoding='utf-8')

## ALGUNS CÓDIGOS QUE PODEM VIR A SER ÚTEIS


In [ ]:
# mapeia cada valor da variável Embarked em um valor numérico
#mapeamento_embarked = {"S": 1, "C": 2, "Q": 3}
#treinamento_df['Embarked'] = treinamento_df['Embarked'].map(mapeamento_embarked)
#teste_df['Embarked'] = teste_df['Embarked'].map(mapeamento_embarked)